# How to handle connections to devices

## Automatically

By default, connections are handled automatically:

In [1]:
from nornir import InitNornir
from nornir_napalm.plugins.tasks import napalm_get

import pprint

In [2]:
nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=napalm_get,
    getters=["facts"]
)
pprint.pprint(r["rtr00"][0].result)

{'facts': {'fqdn': 'localhost',
           'hostname': 'localhost',
           'interface_list': ['Ethernet1',
                              'Ethernet2',
                              'Ethernet3',
                              'Ethernet4',
                              'Management1'],
           'model': 'vEOS',
           'os_version': '4.15.5M-3054042.4155M',
           'serial_number': '',
           'uptime': '...',
           'vendor': 'Arista'}}


## Manually

In some circumstances, you may want to manage connections manually. To do so you can use 
[open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection), [close_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connection), [close_connections](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connections) and [Nornir.close_connections](../ref/api/nornir.rst#nornir.core.Nornir.close_connections). For instance:

In [3]:
def task_manages_connection_manually(task):
    task.host.open_connection("napalm", configuration=task.nornir.config)
    r = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    task.host.close_connection("napalm")
    
nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_connection_manually,
)

# this time the result in position 0 is the connection
print(f"Connection succeeded: {not r['rtr00'][0].failed}")
# and the result in position 1 is the result for napalm_get
pprint.pprint(r["rtr00"][1].result)

Connection succeeded: True
{'facts': {'fqdn': 'localhost',
           'hostname': 'localhost',
           'interface_list': ['Ethernet1',
                              'Ethernet2',
                              'Ethernet3',
                              'Ethernet4',
                              'Management1'],
           'model': 'vEOS',
           'os_version': '4.15.5M-3054042.4155M',
           'serial_number': '',
           'uptime': '...',
           'vendor': 'Arista'}}


## Specifying connection parameters

When using the [open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection) you can specify any parameters you want. If you don't, or if you let nornir open the connection automatically, nornir will read those parameters from the inventory. You can specify standard attributes at the object level if you want to reuse them across different connections or you can override them for each connection. For example:

In [4]:
!sed '2,36!d' ../../tests/inventory_data/hosts.yaml

dev1.group_1:
    port: 65020
    hostname: localhost
    username:
    password: a_password
    platform: eos
    data:
        some_string_to_test_any_all: prefix
        my_var: comes_from_dev1.group_1
        www_server: nginx
        role: www
        nested_data:
            a_dict:
                a: 1
                b: 2
            a_list: [1, 2]
            a_string: asdasd
    groups:
        - group_1
    connection_options:
        paramiko:
            port: 65020
            hostname:
            username: root
            password: docker
            platform: linux
            extras: {}
        dummy:
            hostname: dummy_from_host
            port:
            username:
            password:
            platform:
            extras:
                blah: from_host
